# Regression Week 5: Feature Selection and LASSO (Interpretation)

In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO (using Turi Create, though you can use other solvers). You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second notebook, you will implement your own LASSO solver, using coordinate descent. 

# Fire up Turi Create

In [1]:
import turicreate

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
sales = turicreate.SFrame('home_data.sframe/')

# Create new features

As in Week 2, we consider features that are some transformations of inputs.

In [3]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
sales['floors'] = sales['floors'].astype(float) 
sales['floors_square'] = sales['floors']*sales['floors']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

# Learn regression weights with L1 penalty

Let us fit a model with all the features available, plus the features we just created above.

In [6]:
all_features = ['bedrooms', 'bedrooms_square',
                'bathrooms',
                'sqft_living', 'sqft_living_sqrt',
                'sqft_lot', 'sqft_lot_sqrt',
                'floors', 'floors_square',
                'waterfront', 'view', 'condition', 'grade',
                'sqft_above',
                'sqft_basement',
                'yr_built', 'yr_renovated']

Applying L1 penalty requires adding an extra parameter (`l1_penalty`) to the linear regression call in Turi Create. (Other tools may have separate implementations of LASSO.)  Note that it's important to set `l2_penalty=0` to ensure we don't introduce an additional L2 penalty.

In [55]:
model_all = turicreate.linear_regression.create(sales, target='price', features=all_features,
                                                validation_set=None, 
                                                l2_penalty=0., l1_penalty=1e10, verbose = False)

Find what features had non-zero weight.

In [56]:
model_all.coefficients[model_all.coefficients['value']!= 0].tail()


name,index,value,stderr
(intercept),None,274873.05595049576,None
bathrooms,None,8468.531086910105,None
sqft_living,None,24.42072098244546,None
sqft_living_sqrt,None,350.06055338605546,None
grade,None,842.0680348976231,None
sqft_above,None,20.024722417091112,None


Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection. 

***QUIZ QUESTION***:
According to this list of weights, which of the features have been chosen? 

# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

Be *very* careful that you use seed = 1 to ensure you get the same answer!

In [11]:
(training_and_validation, testing) = sales.random_split(.9,seed=1) # initial train/test split
(training, validation) = training_and_validation.random_split(0.5, seed=1) # split training into train and validate

Next, we write a loop that does the following:
* For `l1_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type `np.logspace(1, 7, num=13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list.
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

When you call `linear_regression.create()` make sure you set `validation_set = None`.

Note: you can turn off the print out of `linear_regression.create()` with `verbose = False`

In [12]:
import numpy as np

In [14]:
RSS = {}
for l1_penalty in np.logspace(1,7,num=13):
  model_all = turicreate.linear_regression.create(training, target='price', features=all_features,validation_set=None,l2_penalty=0.,l1_penalty=l1_penalty, verbose = False)
  predictions = model_all.predict(validation)
  residuals = validation['price'] - predictions
  rss = (residuals*residuals).sum()
  RSS[l1_penalty] = rss

min_rss = min(RSS.values())
min_penalty = [key for key in RSS if RSS[key] == min_rss][0]
print (min_penalty, min_rss)                   

10.0 625766285142460.5


*** QUIZ QUESTION. *** What was the best value for the `l1_penalty`?

In [15]:
model_min_rss = turicreate.linear_regression.create(training, target='price', features=all_features,validation_set=None,l2_penalty=0.,l1_penalty=min_penalty, verbose = False)

***QUIZ QUESTION***
Also, using this value of L1 penalty, how many nonzero weights do you have?

In [53]:
model_min_rss.coefficients[model_min_rss.coefficients['value']!=0]
model_min_rss.coefficients['value'].nnz()

18

# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [17]:
max_nonzeros = 7

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`:

In [18]:
l1_penalty_values = np.logspace(8, 10, num=20)

Now, implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(8, 10, num=20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.
        * *Hint: `model.coefficients['value']` gives you an SArray with the parameters you learned.  If you call the method `.nnz()` on it, you will find the number of non-zero parameters!* 

In [21]:
NNZ = {}
for l1_penalty in np.logspace(8,10,num=20):
  model_all = turicreate.linear_regression.create(training, target='price', features=all_features,validation_set=None,l2_penalty=0.,l1_penalty=l1_penalty, verbose = False)
  NNZ[l1_penalty] = model_all.coefficients['value'].nnz()

NNZ

{100000000.0: 18,
 127427498.57031322: 18,
 162377673.91887242: 18,
 206913808.111479: 18,
 263665089.87303555: 17,
 335981828.6283788: 17,
 428133239.8719396: 17,
 545559478.1168514: 17,
 695192796.1775591: 17,
 885866790.4100832: 16,
 1128837891.6846883: 15,
 1438449888.2876658: 15,
 1832980710.8324375: 13,
 2335721469.0901213: 12,
 2976351441.6313133: 10,
 3792690190.7322536: 6,
 4832930238.571753: 5,
 6158482110.6602545: 3,
 7847599703.514623: 1,
 10000000000.0: 1}

Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzeros` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzeros` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)


*Hint: there are many ways to do this, e.g.:*
* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries.

In [33]:
l1_penalty_min = [key for key in NNZ if NNZ[key] > max_nonzeros]
l1_penalty_min.sort(reverse=True)
l1_penalty_min = l1_penalty_min[0]
l1_penalty_max = [key for key in NNZ if NNZ[key] < max_nonzeros]
l1_penalty_max.sort()
l1_penalty_max = l1_penalty_max[0]

print (l1_penalty_min)
print (l1_penalty_max)

2976351441.6313133
3792690190.7322536


***QUIZ QUESTION.*** What values did you find for `l1_penalty_min` and `l1_penalty_max`, respectively? 

## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [34]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)

* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`.

In [35]:
RSS = {}
for l1_penalty in  l1_penalty_values:
  model_all = turicreate.linear_regression.create(training, target='price', features=all_features,validation_set=None,l2_penalty=0.,l1_penalty=l1_penalty, verbose = False)
  predictions = model_all.predict(validation)
  residuals = validation['price'] - predictions
  rss = (residuals*residuals).sum()
  RSS[l1_penalty] = [rss,model_all.coefficients['value'].nnz()]

RSS

{2976351441.6313133: [966925692362085.2, 10],
 3019316638.9524155: [974019450084554.6, 10],
 3062281836.2735176: [981188367942453.8, 10],
 3105247033.5946198: [989328342459473.5, 10],
 3148212230.915722: [998783211265890.1, 10],
 3191177428.236824: [1008477167020094.4, 10],
 3234142625.557926: [1018298780553819.4, 10],
 3277107822.8790283: [1028247992205977.0, 10],
 3320073020.2001305: [1034616909232827.4, 8],
 3363038217.5212326: [1038554735941040.2, 8],
 3406003414.8423347: [1043237237871701.2, 8],
 3448968612.163437: [1046937488751711.5, 7],
 3491933809.484539: [1051147625612862.5, 7],
 3534899006.805641: [1055992735342999.4, 7],
 3577864204.126743: [1060799531763286.9, 7],
 3620829401.447845: [1065707689498230.1, 6],
 3663794598.768947: [1069464335425585.2, 6],
 3706759796.0900493: [1073504549585599.1, 6],
 3749724993.4111514: [1077632775581415.4, 6],
 3792690190.7322536: [1081867592324112.2, 6]}

In [57]:
min_penalty = 0
best_rss = max(RSS.values())[0]
for key in RSS:
  if (RSS[key][1]==max_nonzeros) and (RSS[key][0]<best_rss):
    best_rss = RSS[key][0]
    min_penalty = key
print(min_penalty)

3448968612.163437


In [58]:
model_all = turicreate.linear_regression.create(training, target='price', features=all_features,validation_set=None,l2_penalty=0.,l1_penalty=min_penalty, verbose = False)

In [59]:
model_all.coefficients[model_all.coefficients['value']!=0]

name,index,value,stderr
(intercept),None,222253.1925443279,None
bedrooms,None,661.7227177822278,None
bathrooms,None,15873.957259268,None
sqft_living,None,32.41022145125974,None
sqft_living_sqrt,None,690.1147733133166,None
grade,None,2899.420269749878,None
sqft_above,None,30.011575302201138,None


***QUIZ QUESTIONS***
1. What value of `l1_penalty` in our narrow range has the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`?
2. What features in this model have non-zero coefficients?